# Machine Learning from scratch

I decided I wanted to review Machine Learning from the bottom up for some personal projects of my own, I realized my education and path in the field so far were missing some fundementals in ways that have been having some consequences in complex problems. I believe that missing out in how the absolute basics are being computed have caused me to make some mistakes that would have been easily avoided had I implemented some of the underlying algorithms from scratch. While I did implement the true "basics" such as Matmuls in undergrad, I never truly covered back propagation at the implementation level (unless you count my very easy vector calc course, which I don't). These resources will seek to build the fundemental ideas of Machine Learning from the ground up while attempting to explain the math at every level. The math involved in the derivation fo these algorithms will be the key focus of these posts, and the code's sole purpose will be to explain the math. This is because during my time reviewing many resources online, I found that a lot of derivations skip some very crucial steps in understanding why many simplifications work, which can lead to erroneous assumptions in how a library might make many computations. The most powerful tools libraries like Torch have, are the autograd systems. However, without a deep understanding of the underlying math, one can easily go down many wrong paths. It is also very easy to dupe oneself into thinking that because they covered the math basics at an introductory level in undergrad, they have a clear picture of what is going on underneath the hood. While it is true that a lot of the math being used is very basic, when applied in the context of Machine Learning, a lot of the finer details can be overlooked.

## The Dot Product and Matrix Multiplication

This doesn't require very much review, and I'll assume anyone reading this understands what "vectors" are, if you don't as a review I will remind you that a vector is just an element of a vector space. Which is a mathematicians way of saying, it's just any element that exists within a larger space of elements. In our case it's numbers which are used to represent a linear function $x_1 + x_2 +x_3 + \cdots + x_n$. In Machine Learning we only ever work with the Real numbers represented as $\mathbb{R}$. Libraries like torch do have some operations that do end up using the Complex numbers for things like computing EigenValues, but that is a bridge to be crossed when we get there.

## The dot product
The dot product is a simple operation, it takes two vectors of length $n$ and it multiplies them together into a single scaler value. If you don't remember what a scaler is, think of it as a constant that can be used to multiply a vector, affecting its magnitude if it's positive and direction if it's negative. It can be described as such: $$a = a_1 + a_2 + a_3 + \cdots + a_n $$ $$b= b_1 + b_2 + b_3 \cdots + b_n$$ $$a \bullet b = a_1 b_1 + a_2 b_2 + a_3 b_3 \cdots a_n b_n $$ $$a \bullet b = \sum_{i=1}^n a_i * b_i$$

Python Implementation:

In [1]:
def dot_product(a,b):
    assert len(a) == len(b)
    res = 0
    for ai,bi in zip(a,b):
        res += ai * bi
    return res

Throughout these notebooks we will also be periodically comparing our implementations to those of many libraries to verify correctness, here is a demonstration using the numpy dot to confirm this is correct

In [2]:
a = [.4,.3,.2,.6]
b = [.1,.5,.6,.9]
import numpy as np


In [3]:
dot_product(a,b)

0.8500000000000001

In [4]:
np.dot(a,b)

0.8500000000000001

In [5]:
assert np.dot(a,b) == dot_product(a,b)

Now that we've demonstrated the math and the code for the dot product we will be using the numpy implementation going forward, this is because it's optimized, part of the purpose of this notebook is to build it from the ground up so we can understand the libraries we regularly rely on.

## Matrix Multiplication

The next step is matrix multiplication. Suppose we have two matrices, a and b, a matmul b is a linear map that produces c, represented as such: $$
\begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}
\end{bmatrix} \bullet \begin{bmatrix}
b_{11} & b_{12} & b_{13}\\
b_{21} & b_{22} & b_{23}\\
b_{31} & b_{32} & b_{33}
\end{bmatrix}  =\begin{bmatrix} a_{11} b_{11} + a_{12} b_{21} + a_{13} b_{31} &  a_{11} b_{12} + a_{12} b_{22} + a_{13} b_{32} &  a_{11} b_{13} + a_{12} b_{23} + a_{13} b_{33} \\
a_{21} b_{11} + a_{22} b_{21} + a_{23} b_{31} & a_{21} b_{12} + a_{22} b_{22} + a_{23} b_{32}& a_{21} b_{13} + a_{22} b_{23} + a_{23} b_{33}\\
a_{31} b_{11} + a_{32} b_{21} + a_{33} b_{31} & a_{31} b_{12} + a_{32} b_{22} + a_{33} b_{32} & a_{31} b_{13} + a_{32} b_{23} + a_{33} b_{33}
\end{bmatrix}  $$

Viewing this, it can be deduced that a matmul is a linear map that takes the dot product of each row in matrix A with each column in matrix B producing an element in matrix C. This means matrix multiplication posseses some very important mathematical features: $$additivity : f(u +v) = f(u) + f(v)$$ and $$homogeneity : f(c * v) = c * f(v) $$ Note: here c is a scaler multiple. 

These properties are very important because they will allow us to simplify our derivations in many places, reducing compute costs. Matrix multiplications tend to hover around $O(n^3)$ complexity, meaning that any algorithm that implements it must make ~$N^3$ computer operations, so allowing us to apply addition and scaler multiplication to them $\textit{after}$ the matmul or even other functions that have the properties of linear maps reduces an insane amount of overhead.

### Some consequences of the Matrix Multiplication

Before diving into the implementation, I'd like to point out that so far we've only actually covered the base cases of matrix multiplication, that is when two matrixes have the same dimensions. The dot product is really just the case where both matrices have a single column. However it is possible to multiply two matrices with slightly different shapes and this is incredibly important, as it is the operation that is used in machine learning to change the dimensions of input features within neural networks, but I am getting ahead of myself. The main thing understand is that if A is a matrix with dimensions: $$A = A_i , A_j$$ and B is a matrix: $$B = B_k , B_l$$ the two matrices can only be multiplied if and only if $$j =k $$
and the resulting matrix will have the dimensions $$C = C_i, C_l$$

#### Non Commutativity and the Determinant property

The last important properties I will mention here is that even though a Matrix Multiplication is a Linear Map, Matrix Multiplication does not have the Commutative property! That means: $$A\bullet B \neq B \bullet A$$ This is extremely important to remember throughout machine learning and even when relying on things like torch autograd, because it means that many operations need to maintain a consistent order to function. Many mistakes and errors can be traced back to simple mistakes involving the order of an operation. The determinant property which we don't need right now simply states that $det(AB) = det(A) det(B)$. I'm mentioning this seperately because even though we covered linear maps earlier, the determinant is actually a multilinear map. We won't be using this property for a while, but it is important to keep in mind.

#### The Transpose of a Matrix
The Transpose is a Linear Map operation that can be applied to a matrix. It is defined as interchanging a matrix's rows and columns. Here is an example:
 $$A= \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}
\end{bmatrix}$$ then $$A^T = \begin{bmatrix}
a_{11} & a_{21} & a_{31}\\
a_{12} & a_{22} & a_{32}\\
a_{13} & a_{23} & a_{33}
\end{bmatrix}$$

##### Implementing Matrix Multiplication

In [20]:
def get_column(matrix,i):
    j = 0
    col = []
    while j < len(matrix):
        col.append(matrix[j][i])
        j += 1
    return col
def get_row(matrix,i):
    j = 0
    row = []
    while j < len(matrix[0]):
        row.append(matrix[i][j])
        j += 1
    return row
def matmul(a,b):
    a_rows,a_cols = len(a), len(a[0])
    b_rows,b_cols = len(b),len(b[0])
    assert a_cols == b_rows
    C = [[0 for i in range(a_rows)] for j in range(b_cols)]
    for i in range(len(C)):
        for j in range(len(C[0])):
            arow = get_row(a,i)
            bcol = get_column(b,j)
            v = 0
            for k,l in zip(arow,bcol):
                v += k * l
            C[i][j] = v
    return C

In [34]:
a = [[1,2,3,7],[3,4,4,10],[7,9,9,10]]
b = [[5,6,5],[7,8,8],[23,27,92],[89,76,123]]
matmul(a,b)


[[711, 635, 1158], [1025, 918, 1645], [1195, 1117, 2165]]

In [35]:
np.matmul(a,b)

array([[ 711,  635, 1158],
       [1025,  918, 1645],
       [1195, 1117, 2165]])

In [31]:
assert np.all(np.array(matmul(a,b)) == np.matmul(a,b))

# Conclusion

Now that we have set up the very basics we can begin implementing linear regression and build from there, in the next section we will use these topics to derive basic linear regression, and from there we can build up to the Multi Layer Perceptron, the heart of Machine Learning.